# 베이즈 정리와 분류 문제

베이즈 정리는 머신 러닝 중 분류(classification) 문제를 해결하는데 사용될 수 있다. 분류 문제는 입력 자료 $X$의 값으로부터 카테고리 값인 출력 자료 $Y$의 값을 예측(prediction)하는 문제이다. 다음과 같은 문제는 분류 문제의 한 예이다.

## 과일 선택 문제

어떤 과일 가게에서 사과와 오렌지를 팔고 있다. 사과와 오렌지는 Red 농장과 Blue 농장에서 공급받는다.

어느날 Red 농장에서 붉은 상자에 사과 2개, 오렌지 6개를 담아서 보냈다.
Blue 농장에서는 푸른 상자에 사과가 3, 오렌지가 1개를 담아서 공급했다.

이 과일을 받은 과일 가게 주인은 이 과일들을 모두 상자에서 꺼내어 늘어놓고 판매를 시작하였다.
이 때 어떤 손님이 와서 오렌지를 하나 선택했다($X=$ 오렌지). 이 오렌지는 Red 농장에서 온 것일까($Y=$Red), 아니면 Blue 농장에서 온 것일까($Y=$Blue)?
만약 Red 농장과 Blue 농장 중 하나의 답을 정해야 한다면 여러분은 어느 농장을 선택하겠는가?

<img src="https://datascienceschool.net/upfiles/41cc9ae1d9c247b68fa6688189d9fa34.png" style="width: 60%">

이 문제의 답은 누구나 직관적으로 알 수 있다. 오렌지가 더 많은 상자가 붉은 상자(Red 농장)이기 때문에 꺼낸 과일이 오렌지라면 상자는 원산지가 Red 농장일 가능성이 높을 것이다. 그 가능성 즉, 확률을 수치화해보자.

이 문제를 확률론의 용어로 다시 정리한다.

* 손님이 선택한 과일이 사과인 사건 $X=A$
* 손님이 선택한 과일이 오렌지인 사건 $X=O$
* 손님이 선택한 과일이 Red 농장에서 왔을 사건 $Y=R$
* 손님이 선택한 과일이 Blue 농장에서 왔을 사건 $Y=B$

우리가 원하는 확률은 "손님이 오렌지를 선택한 경우에 원산지가 Red 농장일 확률"이므로 다음과 같은 조건부 확률이다.

$$ P(Y=R|X=O)  $$

이 조건부 확률 값을 구하기 위해 우선 다음과 같은 확률 값을 구한다.

전체 샘플 12개 중에서 Red 농장에서 온 샘플이 8개이므로

$$ P(Y=R) = \dfrac{8}{8 + 4}  = \dfrac{8}{12} $$

Red 농장의 샘플 8개 중에서 오렌지가 6개이므로

$$ P(X=O | Y=R) = \dfrac{6}{2 + 6}  = \dfrac{6}{8} $$

이 값과 베이즈 정리를 이용하여 조건부 확률을 계산하면 다음과 같다.

$$ 
\begin{eqnarray}
P(Y=R|X=O) 
&=& \dfrac{P(X=O|Y=R)P(Y=R)}{P(X=O)} \\
&=& \dfrac{P(X=O|Y=R)P(Y=R)}{P(X=O|Y=R)P(Y=R) + P(X=O|Y=B)P(Y=B)} \\
&=& \dfrac{6/8 \cdot 8/12}{6/8 \cdot 8/12 + 1/4 \cdot 4/12 } \\
&=& \dfrac{1/2}{1/2 + 1/12} = 6/7\\
\end{eqnarray}
$$


농장이 두 개 뿐이므로 원상지가 Blue 농장인 확률은 $ 1 - P(Y=R|X=O) $이다.


 $$ P(Y=B|X=O) = 1 - P(Y=R|X=O) = 1/7 $$

답이 Red 농장일 확률이 $6/7$, Blue 농장일 확률이 $1/7$이므로 예측 결과는 Red 농장이다.

이러한 방식으로 분류 문제를 풀기 위해서는 각각의 출력 카테고리 $Y$ 에 대한 특징값 $X$의 분포, 즉 likelihood를 알고 있어야 한다. 이렇게 베이즈 정리와 likelihood를 이용하여 각각의 $Y$ 값에 대한 확률 값을 모두 구한 다음, 가장 확률 값이 높은 $Y$값을 선택하여 분류 문제를 푸는 방법을 생성론적 방법(generative method)라고 한다.

pgmpy를 사용하여 위 문제를 풀면 다음과 같다.

In [2]:
from pgmpy.factors.discrete import TabularCPD

여기에서는 확률 변수 Y를 Y=0인 경우가 Red 농장, Y=1인 경우가 Blue 농장으로 정의한다.

In [3]:
cpd_Y = TabularCPD('Y', 2, [[8/12, 4/12]])
print(cpd_Y)

+-----+----------+
| Y_0 | 0.666667 |
+-----+----------+
| Y_1 | 0.333333 |
+-----+----------+


확률 변수 X는 X=0인 경우를 오렌지인 경우, X=1인 경우를 사과인 경우으로 정의한다.

In [4]:
cpd_XY = TabularCPD('X', 2, np.array([[6/8, 1/4], [2/8, 3/4]]),
                    evidence=['Y'], evidence_card=[2])
print(cpd_XY)

+-----+------+------+
| Y   | Y_0  | Y_1  |
+-----+------+------+
| X_0 | 0.75 | 0.25 |
+-----+------+------+
| X_1 | 0.25 | 0.75 |
+-----+------+------+


베이지안 네트워크 모형으로 사후확률을 계산하면 $0.1429 = \frac{1}{7}$이 나온다.

In [7]:
from pgmpy.models import BayesianModel

model = BayesianModel([('Y', 'X')])
model.add_cpds(cpd_Y, cpd_XY)
model.check_model()

True

In [9]:
from pgmpy.inference import VariableElimination

infer = VariableElimination(model)
posterior = infer.query(['Y'], evidence={'X': 0})
print(posterior['Y'])

+-----+----------+
| Y   |   phi(Y) |
+=====+==========+
| Y_0 |   0.8571 |
+-----+----------+
| Y_1 |   0.1429 |
+-----+----------+


In [10]:
1/7

0.14285714285714285